In [358]:
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell

In [359]:
InteractiveShell.ast_node_interactivity = "all"

In [360]:
student_data = pd.read_csv('students_complete.csv')

In [362]:
school_data = pd.read_csv('schools_complete.csv')
school_data.sort_values("name", inplace=True)
school_data.head()

,School ID,name,type,size,budget
7,7,Bailey High School,District,4976,3124928
6,6,Cabrera High School,Charter,1858,1081356
1,1,Figueroa High School,District,2949,1884411
13,13,Ford High School,District,2739,1763916
4,4,Griffin High School,Charter,1468,917500


In [363]:
Total_School = school_data.shape[0]
Total_Student = student_data.shape[0]
Total_Budget = school_data["budget"].sum()
Avg_Math = student_data["math_score"].mean()
Avg_Reading = student_data["reading_score"].mean()
Per_Total_MPass = student_data[student_data["math_score"]> 70 ]["math_score"].count()/Total_Student*100
Per_Total_RPass = student_data[student_data["reading_score"]> 70]["reading_score"].count()/Total_Student*100
Per_Total_Pass = (Per_Total_MPass + Per_Total_RPass)/2

In [364]:
District_Summary = pd.DataFrame({"Total Schools":Total_School, "Total Students": "{:,}".format(Total_Student), "Total Budget": "$"+"{:,}".format(Total_Budget), 
                                 "Average Math Score": Avg_Math, "Average Reading Score": Avg_Reading, "% Passing Math": Per_Total_MPass
                                 ,"% Passing Reading": Per_Total_RPass, "% Passing": Per_Total_Pass},index = [0])
District_Summary = District_Summary[["Total Schools", "Total Students", "Total Budget", "Average Math Score","Average Reading Score",
                   "% Passing Math","% Passing Reading", "% Passing"]]
District_Summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing
0,15,"39,170","$24,649,428",78.985371,81.87784,72.392137,82.971662,77.681899


In [365]:
student_data.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [366]:
school_data_complete = pd.merge(student_data, school_data, how="left", left_on="school", right_on="name")

In [367]:
school_data_complete.head()

,Student ID,name_x,gender,grade,school,reading_score,math_score,School ID,name_y,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,Huang High School,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,Huang High School,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,Huang High School,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,Huang High School,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,Huang High School,District,2917,1910635


In [368]:
per_school_counts = school_data_complete["name_y"].value_counts()

In [369]:
school_passing_math = school_data_complete[(school_data_complete["math_score"] > 70)]

In [370]:
school_passing_reading = school_data_complete[(school_data_complete["reading_score"] > 70)]

In [371]:
per_school_passing_math = school_passing_math.groupby(["name_y"]).count()["name_x"] / per_school_counts * 100
per_mpass_df = pd.DataFrame({"% Passing Math": per_school_passing_math.values}, index = per_school_passing_math.index)

In [373]:
per_school_passing_reading = school_passing_reading.groupby(["name_y"]).count()["name_x"] / per_school_counts * 100
per_rpass_df = pd.DataFrame({"% Pass Reading": per_school_passing_reading.values}, index = per_school_passing_reading.index)

In [375]:
per_school_pass = (per_school_passing_math + per_school_passing_reading) / 2
per_pass_df = pd.DataFrame({"Overall % Passing": per_school_pass.values}, index = per_school_pass.index)

In [376]:
School_Pt1 = pd.pivot_table(school_data_complete, values = "name_x", index = "school", 
                            aggfunc = lambda x: len(x.unique()))
School_Pt2 = school_data[["budget", "name"]]
School_Pt2.set_index("name", inplace = True)
School_Pt3 = pd.pivot_table(school_data_complete, values = ["math_score","reading_score"], index = "school")
School_Summary = School_Pt1.merge(School_Pt2, left_index = True, 
                                right_index = True).merge(School_Pt3, left_index = True, 
                                right_index = True).merge(per_mpass_df, left_index = True, right_index = True).merge(
                                per_rpass_df, left_index = True, right_index = True).merge(per_pass_df, left_index = True, right_index = True)

In [377]:
School_Summary["Per Student Budget"] = School_Summary["budget"]/ School_Summary["name_x"]
school_type = school_data["type"].tolist()
School_Summary["Type"] = school_type

In [413]:
School_Summary = School_Summary.rename(columns={"name_x":"Total Students", "budget":"Total School Budget", "math_score":"Avg. Math Score",
                              "reading_score":"Avg. Reading Score"})
School_Summary = School_Summary[["Type","Total Students", "Total School Budget","Per Student Budget","Avg. Math Score", "Avg. Reading Score","% Passing Math",
                                 "% Pass Reading","Overall % Passing"]]
School_Summary.head()

,Type,Total Students,Total School Budget,Per Student Budget,Avg. Math Score,Avg. Reading Score,% Passing Math,% Pass Reading,Overall % Passing
school,,,,,,,,,
Bailey High School,District,4799,3124928,651.162325,77.048432,81.033963,64.630225,79.300643,71.965434
Cabrera High School,Charter,1834,1081356,589.616140,83.061895,83.975780,89.558665,93.864370,91.711518
Figueroa High School,District,2886,1884411,652.949064,76.711767,81.158020,63.750424,78.433367,71.091896
Ford High School,District,2676,1763916,659.161435,77.102592,80.746258,65.753925,77.510040,71.631982
Griffin High School,Charter,1449,917500,633.195307,83.351499,83.816757,89.713896,93.392371,91.553134


In [391]:
Best_Schools = School_Summary.sort_values(by="Overall % Passing", ascending= False)
Best_Schools.head()

,Type,Total Students,Total School Budget,Per Student Budget,Avg. Math Score,Avg. Reading Score,% Passing Math,% Pass Reading,Overall % Passing
school,,,,,,,,,
Wilson High School,Charter,2248,1319574,586.999110,83.274201,83.989488,90.932983,93.254490,92.093736
Pena High School,Charter,955,585858,613.463874,83.839917,84.044699,91.683992,92.203742,91.943867
Wright High School,Charter,1783,1049400,588.558609,83.682222,83.955000,90.277778,93.444444,91.861111
Cabrera High School,Charter,1834,1081356,589.616140,83.061895,83.975780,89.558665,93.864370,91.711518
Holden High School,Charter,424,248087,585.110849,83.803279,83.814988,90.632319,92.740047,91.686183


In [392]:
Bottom_Schools = School_Summary.sort_values(by="Overall % Passing")
Bottom_Schools.head()

,Type,Total Students,Total School Budget,Per Student Budget,Avg. Math Score,Avg. Reading Score,% Passing Math,% Pass Reading,Overall % Passing
school,,,,,,,,,
Rodriguez High School,District,3902,2547363,652.835213,76.842711,80.744686,64.066017,77.744436,70.905226
Huang High School,District,2858,1910635,668.521693,76.629414,81.182722,63.318478,78.813850,71.066164
Johnson High School,District,4618,3094650,670.127761,77.072464,80.966394,63.852132,78.281874,71.067003
Figueroa High School,District,2886,1884411,652.949064,76.711767,81.158020,63.750424,78.433367,71.091896
Hernandez High School,District,4501,3022020,671.410798,77.289752,80.934412,64.746494,78.187702,71.467098


In [393]:
Math_Score_ByGrade = school_data_complete[["school","grade","math_score"]]
MStable = pd.pivot_table(Math_Score_ByGrade, values = "math_score", index = "school", columns = "grade" )
MStable[["9th", "10th", "11th", "12th"]]

grade,9th,10th,11th,12th
school,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [336]:
Reading_Score_ByGrade = school_data_complete[["school","grade","reading_score"]]
RStable = pd.pivot_table(Reading_Score_ByGrade, values = "reading_score", index = "school", columns = "grade" )
RStable[["9th", "10th", "11th", "12th"]]

grade,9th,10th,11th,12th
school,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [415]:
bins = [0,600, 625, 650,675]
size_bins = [0,1000,2000,5000]
bin_labels = ["< $600", "$600-625","$625-650","$650-675"]
size_labels = ["Small (<1000)", "Medium (1000-2000)","Large (2000-500)"]
School_Summary["Spending Ranges (Per Student)"] = pd.cut(School_Summary["Per Student Budget"],bins,labels=bin_labels)
School_Summary["School Size"] = pd.cut(School_Summary["Total Students"],size_bins,labels=size_labels)
School_Summary.head()

,Type,Total Students,Total School Budget,Per Student Budget,Avg. Math Score,Avg. Reading Score,% Passing Math,% Pass Reading,Overall % Passing,Spending Ranges (Per Student),School Size
school,,,,,,,,,,,
Bailey High School,District,4799,3124928,651.162325,77.048432,81.033963,64.630225,79.300643,71.965434,$650-675,Large (2000-500)
Cabrera High School,Charter,1834,1081356,589.616140,83.061895,83.975780,89.558665,93.864370,91.711518,< $600,Medium (1000-2000)
Figueroa High School,District,2886,1884411,652.949064,76.711767,81.158020,63.750424,78.433367,71.091896,$650-675,Large (2000-500)
Ford High School,District,2676,1763916,659.161435,77.102592,80.746258,65.753925,77.510040,71.631982,$650-675,Large (2000-500)
Griffin High School,Charter,1449,917500,633.195307,83.351499,83.816757,89.713896,93.392371,91.553134,$625-650,Medium (1000-2000)


In [409]:
Score_by_Spending = pd.pivot_table(School_Summary, values = ["Avg. Math Score", "Avg. Reading Score","% Passing Math",
                                 "% Pass Reading","Overall % Passing"], index = "Spending Ranges (Per Student)")
Score_by_Spending

,% Pass Reading,% Passing Math,Avg. Math Score,Avg. Reading Score,Overall % Passing
Spending Ranges (Per Student),,,,,
< $600,93.325838,90.350436,83.455399,83.933814,91.838137
$600-625,92.410786,90.788049,83.599686,83.885211,91.599418
$625-650,93.148785,89.963982,83.384924,83.832844,91.556383
$650-675,78.324559,64.302528,76.956733,80.966636,71.313543


In [410]:
Score_by_School_Size = pd.pivot_table(School_Summary, values = ["Avg. Math Score", "Avg. Reading Score","% Passing Math",
                                 "% Pass Reading","Overall % Passing"], index = "School Size")
Score_by_School_Size

,% Pass Reading,% Passing Math,Avg. Math Score,Avg. Reading Score,Overall % Passing
School Size,,,,,
Small (<1000),92.471895,91.158155,83.821598,83.929843,91.815025
Medium (1000-2000),93.244843,89.931303,83.374684,83.864438,91.588073
Large (2000-500),80.190800,67.631335,77.746417,81.344493,73.911067


In [412]:
Score_by_type = Score_by_Spending = pd.pivot_table(School_Summary, values = ["Avg. Math Score", "Avg. Reading Score","% Passing Math",
                                 "% Pass Reading","Overall % Passing"], index = "Type")
Score_by_type

,% Pass Reading,% Passing Math,Avg. Math Score,Avg. Reading Score,Overall % Passing
Type,,,,,
Charter,93.052812,90.363226,83.473852,83.896421,91.708019
District,78.324559,64.302528,76.956733,80.966636,71.313543
